In [ ]:
import numpy as np
import pandas as pd
import glob
import os

## Load files from Drive and Set up Working Directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Copy folder from Drive to Colab directory, takes 30 seconds
!cp -r /content/drive/MyDrive/uav_palm_data/train2017 /content
!cp -r /content/drive/MyDrive/uav_palm_data/val2017 /content

# Unzip image folders
!unzip -q /content/train2017/train2017.zip -d /content/train2017/ 
!unzip -q /content/val2017/val2017.zip -d /content/val2017/

# Create folder structure
!mkdir /content/train
!mkdir /content/test
!mkdir /content/train/images
!mkdir /content/train/labels
!mkdir /content/test/images
!mkdir /content/test/labels

# Move train2017 and val2017 from Drive to Colab folders
!mv /content/train2017/train2017/*.jpg /content/train/images/
!mv /content/val2017/val2017/*.jpg /content/test/images/

In [ ]:
!git clone https://github.com/ultralytics/JSON2YOLO

Cloning into 'JSON2YOLO'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 262 (delta 92), reused 79 (delta 79), pack-reused 164
Receiving objects: 100% (262/262), 69.61 KiB | 17.40 MiB/s, done.
Resolving deltas: 100% (167/167), done.


In [ ]:
import sys
sys.path.append('/content/JSON2YOLO')
from general_json2yolo import convert_coco_json

In [ ]:
# Convert train2017.json to txt files
convert_coco_json('/content/train2017')

# Move txt files to labels folders
!mv /content/new_dir/labels/train2017/*.txt /content/train/labels
!rm -rf /content/new_dir

# Convert val2017.json to txt files
convert_coco_json('/content/val2017')

# Move txt files to labels folders
!mv /content/new_dir/labels/val2017/*.txt /content/test/labels
!rm -rf /content/new_dir

Annotations /content/train2017/instances_train2017.json: 100%|██████████| 262811/262811 [00:18<00:00, 14457.62it/s]
Annotations /content/val2017/instances_val2017.json: 100%|██████████| 32846/32846 [00:02<00:00, 14505.95it/s]


## Separate training file into train and validation set

In [ ]:
list_image = []
for f in glob.glob('/content/train/labels/*'):
  list_image.append(f)

In [ ]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(list_image, test_size=0.3, random_state=11)

In [ ]:
len(val)

1200

In [ ]:
!mkdir /content/uav_palm_data
!mkdir /content/uav_palm_data/images
!mkdir /content/uav_palm_data/images/train
!mkdir /content/uav_palm_data/images/val
!mkdir /content/uav_palm_data/labels
!mkdir /content/uav_palm_data/labels/train
!mkdir /content/uav_palm_data/labels/val

In [ ]:
image_dest = '/content/uav_palm_data/images/train'
label_dest = '/content/uav_palm_data/labels/train'

for t in train:
  fileid = t.split('/')[-1][:-4]
  image_name = fileid + '.jpg'
  label_name = fileid + '.txt'
  image_src = '/content/train/images/' + image_name
  label_src = '/content/train/labels/' + label_name
  os.system(f'mv {image_src} {image_dest}')
  os.system(f'mv {label_src} {label_dest}')


In [ ]:
image_dest = '/content/uav_palm_data/images/val'
label_dest = '/content/uav_palm_data/labels/val'

for v in val:
  fileid = v.split('/')[-1][:-4]
  image_name = fileid + '.jpg'
  label_name = fileid + '.txt'
  image_src = '/content/train/images/' + image_name
  label_src = '/content/train/labels/' + label_name
  os.system(f'mv {image_src} {image_dest}')
  os.system(f'mv {label_src} {label_dest}')
  

## Run YOLO

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
from yolov5 import utils

Cloning into 'yolov5'...
remote: Enumerating objects: 12262, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12262 (delta 2), reused 6 (delta 1), pack-reused 12250
Receiving objects: 100% (12262/12262), 12.00 MiB | 30.88 MiB/s, done.
Resolving deltas: 100% (8488/8488), done.
/content/yolov5
     |████████████████████████████████| 596 kB 5.0 MB/s 


In [ ]:
%%writefile /content/yolov5/palm.yaml
train: /content/uav_palm_data/images/train
val: /content/uav_palm_data/images/val
nc: 5
names: ["palm1", "palm2", "palm3", "palm4", "palm5"]

Writing /content/yolov5/palm.yaml


In [ ]:
!python train.py --img 1000 --batch 16 --epochs 50 --data palm.yaml --weights yolov5s.pt --save-period 3

train: weights=yolov5s.pt, cfg=, data=palm.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=1000, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=3, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-258-g1156a32 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.

In [ ]:
# !zip -r '/content/runs_ieee_yolo.zip' '/content/yolov5/runs/train/exp'